In [3]:
# import fast ai library
# https://docs.fast.ai/quick_start.html
from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *

In [4]:
URLs.PETS

'https://s3.amazonaws.com/fast-ai-imageclas/oxford-iiit-pet.tgz'

In [7]:
# import data 
path = untar_data(URLs.PETS)/'images'

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:


def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

In [ ]:
# fine tune a model 

learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:
# predict with the model
img = PILImage.create('images/cat.jpg')
img

is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

**Movie Recommendations**

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")

print("Path to dataset files:", path)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Path to dataset files: /kaggle/input/movielens-100k-dataset


In [8]:
import pandas as pd
path = '/kaggle/input/movielens-100k-dataset/ml-100k'

In [15]:
ratings = pd.read_csv(f'{path}/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


join data with movie & user details

In [16]:
movie_details = pd.read_csv(f'{path}/u.item', sep='|', encoding='utf-8', encoding_errors='ignore', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western'])
movie_details.head()

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,childrens,...,fantasy,film_noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
user_info = pd.read_csv(f'{path}/u.user', sep='|', encoding='utf-8', encoding_errors='ignore', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])
user_info.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [20]:
ratings_all = ratings.merge(movie_details, how='left', left_on='item_id', right_on='movie_id', suffixes=(False, False))
ratings_all.head()

,user_id,item_id,rating,timestamp,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,...,fantasy,film_noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,196,242,3,881250949,242,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,302,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Confidential+(1997),0,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,377,Heavyweights (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Heavyweights%20(1994),0,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,51,Legends of the Fall (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Legends%20of%20the%20Fall%20(1994),0,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,346,Jackie Brown (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-119396,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
ratings_all = ratings_all.merge(user_info, how='left', left_on='user_id', right_on='user_id', suffixes=(False, False))


In [24]:
ratings_all.head()

,user_id,item_id,rating,timestamp,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,...,mystery,romance,sci-fi,thriller,war,western,age,gender,occupation,zip_code
0,196,242,3,881250949,242,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,...,0,0,0,0,0,0,49,M,writer,55105
1,186,302,3,891717742,302,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Confidential+(1997),0,...,1,0,0,1,0,0,39,F,executive,00000
2,22,377,1,878887116,377,Heavyweights (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Heavyweights%20(1994),0,...,0,0,0,0,0,0,25,M,writer,40206
3,244,51,2,880606923,51,Legends of the Fall (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Legends%20of%20the%20Fall%20(1994),0,...,0,1,0,0,1,1,28,M,technician,80525
4,166,346,1,886397596,346,Jackie Brown (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?imdb-title-119396,0,...,0,0,0,0,0,0,47,M,educator,55113


In [30]:
ra = ratings_all.drop(columns=['movie_id', 'imdb_url'])

In [31]:
ra.head()

,user_id,item_id,rating,timestamp,movie_title,release_date,video_release_date,unknown,action,adventure,...,mystery,romance,sci-fi,thriller,war,western,age,gender,occupation,zip_code
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,0,0,0,...,0,0,0,0,0,0,49,M,writer,55105
1,186,302,3,891717742,L.A. Confidential (1997),01-Jan-1997,NaN,0,0,0,...,1,0,0,1,0,0,39,F,executive,00000
2,22,377,1,878887116,Heavyweights (1994),01-Jan-1994,NaN,0,0,0,...,0,0,0,0,0,0,25,M,writer,40206
3,244,51,2,880606923,Legends of the Fall (1994),01-Jan-1994,NaN,0,0,0,...,0,1,0,0,1,1,28,M,technician,80525
4,166,346,1,886397596,Jackie Brown (1997),01-Jan-1997,NaN,0,0,0,...,0,0,0,0,0,0,47,M,educator,55113


In [32]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ra, test_size=0.2)

In [33]:
dls = CollabDataLoaders.from_df(train)
dls.show_batch()

,user_id,item_id,rating
0,18,494,3
1,345,568,4
2,280,1035,4
3,592,762,5
4,618,79,5
5,222,270,2
6,275,62,3
7,798,133,3
8,102,363,2
9,503,226,5


In [34]:
learn = collab_learner(dls, y_range=(0.5,5.5))
learn.fine_tune(6)
# learn.fit_one_cyle(1)


epoch,train_loss,valid_loss,time
0,0.981238,0.998930,00:09


epoch,train_loss,valid_loss,time
0,0.859606,0.945484,00:09
1,0.802642,0.895875,00:09
2,0.716189,0.870370,00:09
3,0.655791,0.858808,00:09
4,0.616367,0.854360,00:09
5,0.633689,0.853849,00:09


In [35]:
learn.show_results()

,user_id,item_id,rating,rating_pred
0,200,768,4,3.728457
1,927,501,4,3.688640
2,452,285,3,3.802979
3,183,94,3,3.013612
4,598,258,5,3.465091
5,892,781,4,3.299072
6,92,231,3,2.204476
7,244,114,4,4.683532
8,178,196,4,4.146698


In [48]:
test_dls = CollabDataLoaders.from_df(test)
# test_dls.head()
preds, y = learn.get_preds(dl = test_dls)


In [46]:
# acc = accuracy(preds, y)
print(y)

tensor([[5],
        [4],
        [4],
        ...,
        [3],
        [2],
        [2]], dtype=torch.int8)
